**Initialization**

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col


**Read Bronze Table**

In [0]:
df = spark.read.table("workspace.bronze.erp_cust_az12")


**Silver Transformation**

**Trim all String values**

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))
     


**Customer ID clean up**

In [0]:
df = df.withColumn(
    "cid",
    F.when(
        col("cid").startswith("NAS"),
        F.substring(col("cid"), 4, F.length(col("cid")))
    ).otherwise(col("cid"))
)


**Birthday Validation**

In [0]:
df = df.withColumn(
     "bdate",
     F.when(col("bdate") > F.current_date(), None) 
      .otherwise(col("bdate")) 
)

**Gender Normalization**

In [0]:
df = df.withColumn(
    "gen",
    F.when(F.upper(col("gen")).isin("F", "FEMALE"), "Female")
     .when(F.upper(col("gen")).isin("M", "MALE"), "Male")
     .otherwise("Other")
)


**Renaming Columns**

In [0]:
df = (
    df
    .withColumnRenamed("cid", "customer_number")
    .withColumnRenamed("bdate", "birth_date")
    .withColumnRenamed("gen", "gender")
)

**Sanity checks of dataframe**

In [0]:
df.limit(10).display()

**Writing Silver Table**

In [0]:
df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("workspace.silver.erp_customers")

**Sanity checks of silver table**

In [0]:
%sql
SELECT * FROM WORKSPACE.silver.erp_customers LIMIT 10;